# DEMO - Training and testing HerdNet on nadir aerial images

## Installations

In [1]:
# Check GPU
# !nvidia-smi

In [2]:
# # Install the dependencies
# !pip install albumentations==1.0.3
# !pip install fiftyone==0.14.3
# !pip install hydra-core==1.1.0
# !pip install opencv-python==4.5.1.48
# !pip install pandas==1.2.3
# !pip install pillow==8.2.0
# !pip install scikit-image==0.18.1
# !pip install scikit-learn==1.0.2
# !pip install scipy==1.6.2
# !pip install wandb==0.10.33

In [3]:
# Download and install the code
import sys
# 
# !git clone https://github.com/Alexandre-Delplanque/HerdNet
# !cd '/content/HerdNet' && python setup.py install
# 
# sys.path.append('/content/HerdNet')

## Create datasets

In [4]:
# Download some of the data of Delplanque et al. (2021) as an example
# !gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O ./data/data.zip
# !unzip -oq "/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/My Drive/Datasets/HerdNet/data.zip" -d ../

In [5]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

/Users/christian/opt/anaconda3/envs/HerdNet310_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Create validation patches using the patcher tool (for demo)
from animaloc.utils.useful_funcs import mkdir

mkdir('../data/val_patches')
!python ../tools/patcher.py ../data/val 512 512 0 ../data/val_patches -csv ../data/val.csv -min 0.0 -all False

Exporting patches: 100%|██████████████████████| 111/111 [01:15<00:00,  1.48it/s]


In [7]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

patch_size = 512
num_classes = 7
down_ratio = 2

train_dataset = CSVDataset(
    csv_file = '../data/train_patches.csv',
    root_dir = '../data/train_patches',
    albu_transforms = [
        A.VerticalFlip(p=0.5), 
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
        A.Blur(blur_limit=15, p=0.2),
        A.Normalize(p=1.0)
        ],
    end_transforms = [MultiTransformsWrapper([
        FIDT(num_classes=num_classes, down_ratio=down_ratio),
        PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
        ])]
    )

val_dataset = CSVDataset(
    csv_file = '../data/val_patches/gt.csv',
    root_dir = '../data/val_patches',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(
    csv_file = '../data/test.csv',
    root_dir = '../data/test',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

In [8]:
# Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset = train_dataset, batch_size = 4, shuffle = True)

val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Define HerdNet for training

In [11]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio)

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0])

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

## Create the Trainer

In [12]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '../output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 100

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet, 
    size=(patch_size,patch_size), 
    overlap=160, 
    down_ratio=down_ratio, 
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=val_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=work_dir, 
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

AssertionError: Torch not compiled with CUDA enabled

## Start training

In [ ]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score')

## Test the model

In [ ]:
# Path to your .pth file
import gdown

pth_path = ''

if not pth_path:
    gdown.download(
        'https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_',
        '/content/20220413_herdnet_model.pth'
        )
    pth_path = '/content/20220413_herdnet_model.pth'

In [ ]:
# Create output folder
test_dir = '/content/test_output'
mkdir(test_dir)

In [ ]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path=pth_path)

In [ ]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=test_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=test_dir, 
    header='test'
    )

In [ ]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

In [ ]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

In [ ]:
# Get the detections
detections = test_evaluator.results
detections